- Gestión dinámica de inventario día a día (físico y digital).
- Política de reabastecimiento up-to-level (Q_i, T).
- Cálculo de costos y métricas diarias.
- Modelo de aceptación de despacho express.


## Carga de Datos Reales y Configuración Inicial

In [103]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [104]:
# Función para cargar ventas
def cargar_ventas(ruta_base):
    ventas = []
    for dia in range(1, 11):
        archivo = f"{ruta_base}_{dia}_20250115.csv"
        if os.path.exists(archivo):
            df = pd.read_csv(archivo)
            df['dia'] = dia
            ventas.append(df)
    return pd.concat(ventas, ignore_index=True)

# Carga de datos únicos
flota = pd.read_csv('flota_20250115.csv')
productos = pd.read_csv('productos_20250115.csv')
proporcion_eleccion = pd.read_csv('proporcion_eleccion_de_usuarios_20250115.csv')
#demanda_insatisfecha = pd.read_csv('demanda_online_insatisfecha_20250115.csv')
reorden = pd.read_csv('reorden_20250115.csv')
tiendas = pd.read_csv('tiendas_20250115.csv')
vehiculos = pd.read_csv('vehiculos_20250115.csv')
zonas = pd.read_csv('zonas_20250115.csv')

# Carga de ventas físicas y digitales
ventas_tienda = cargar_ventas('venta_tienda')
ventas_tienda.rename(columns={'venta_tienda': 'cantidad'}, inplace=True)

ventas_zona = cargar_ventas('venta_zona')
ventas_zona.rename(columns={'venta_digital': 'cantidad'}, inplace=True)

costo_inventario_unitario = 3.733

Generar resumen tiendas con parametros:

In [105]:
from scipy.stats import norm, lognorm, gamma, poisson
import numpy as np
import pandas as pd

# === 1. Usar SOLO ventas físicas ===
ventas_fisicas = ventas_tienda[['id_producto', 'cantidad']]

# === 2. Ajustar distribuciones por producto ===
parametros_tienda = []

for id_producto, grupo in ventas_fisicas.groupby("id_producto"):
    cantidades = grupo["cantidad"].values
    positivas = cantidades[cantidades > 0]

    # Normal
    mu, sigma = norm.fit(cantidades)
    parametros_tienda.append({
        'id_producto': id_producto,
        'distribucion': 'Normal',
        'parametro1': mu,
        'parametro2': sigma
    })

    # Log-Normal
    if len(positivas) > 0:
        try:
            shape, loc, scale = lognorm.fit(positivas, floc=0)
            mu_ln = np.log(scale)
            sigma_ln = shape
            parametros_tienda.append({
                'id_producto': id_producto,
                'distribucion': 'Log-Normal',
                'parametro1': mu_ln,
                'parametro2': sigma_ln
            })
        except:
            pass

    # Gamma
    if len(positivas) > 0:
        try:
            shape_g, loc_g, scale_g = gamma.fit(positivas, floc=0)
            parametros_tienda.append({
                'id_producto': id_producto,
                'distribucion': 'Gamma',
                'parametro1': shape_g,
                'parametro2': scale_g
            })
        except:
            pass

    # Poisson
    lambda_p = cantidades.mean()
    parametros_tienda.append({
        'id_producto': id_producto,
        'distribucion': 'Poisson',
        'parametro1': lambda_p,
        'parametro2': None
    })

df_param_tienda = pd.DataFrame(parametros_tienda)

# === 3. Fusionar con archivo resumen original (para tomar el mejor ajuste) ===
resumen_tienda = pd.read_excel('resumen_tiendas.xlsx')

df_tienda_completo = resumen_tienda.merge(
    df_param_tienda,
    left_on=['id_producto', 'mejor_ajuste'],
    right_on=['id_producto', 'distribucion'],
    how='left'
)

df_tienda_completo.drop(columns=['distribucion'], inplace=True)

# === 4. Guardar Excel final ===
df_tienda_completo.to_excel('resumen_tiendas_con_parametros.xlsx', index=False)


Generar resumen zonas con parametros:

In [106]:
# === 1. Ajuste sobre ventas digitales (zonas) ===
parametros_zona = []

for id_producto, grupo in ventas_zona.groupby("id_producto"):
    cantidades = grupo["cantidad"].values
    positivas = cantidades[cantidades > 0]

    # Normal
    mu, sigma = norm.fit(cantidades)
    parametros_zona.append({'id_producto': id_producto, 'distribucion': 'Normal', 'parametro1': mu, 'parametro2': sigma})

    # Log-Normal
    if len(positivas) > 0:
        try:
            shape, loc, scale = lognorm.fit(positivas, floc=0)
            mu_ln = np.log(scale)
            sigma_ln = shape
            parametros_zona.append({
                'id_producto': id_producto,
                'distribucion': 'Log-Normal',
                'parametro1': mu_ln,
                'parametro2': sigma_ln
            })
        except:
            pass

    # Gamma
    if len(positivas) > 0:
        try:
            shape_g, loc_g, scale_g = gamma.fit(positivas, floc=0)
            parametros_zona.append({
                'id_producto': id_producto,
                'distribucion': 'Gamma',
                'parametro1': shape_g,
                'parametro2': scale_g
            })
        except:
            pass

    # Poisson
    lambda_p = cantidades.mean()
    parametros_zona.append({'id_producto': id_producto, 'distribucion': 'Poisson', 'parametro1': lambda_p, 'parametro2': None})

df_param_zona = pd.DataFrame(parametros_zona)

# === 2. Fusionar con el resumen original de zonas
resumen_zona = pd.read_excel('resumen_zonas.xlsx')

df_zona_completo = resumen_zona.merge(
    df_param_zona,
    left_on=['id_producto', 'mejor_ajuste'],
    right_on=['id_producto', 'distribucion'],
    how='left'
)

df_zona_completo.drop(columns=['distribucion'], inplace=True)

# === 3. Guardar
df_zona_completo.to_excel('resumen_zonas_con_parametros.xlsx', index=False)


c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_continuous_distns.py:3474: RuntimeWarning: divide by zero encountered in scalar divide
  aest = (3-s + np.sqrt((s-3)**2 + 24*s)) / (12*s)
c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_continuous_distns.py:3477: RuntimeWarning: invalid value encountered in scalar subtract
  a = optimize.brentq(lambda a: np.log(a) - sc.digamma(a) - s,


GENERAR DEMANDA

In [107]:
import pandas as pd
import numpy as np
from scipy.stats import norm, lognorm, gamma, poisson
import os

# Leer los parámetros por producto
resumen_zonas_con_param = pd.read_excel('resumen_zonas_con_parametros.xlsx')
resumen_zonas_con_param.set_index('id_producto', inplace=True)

def generar_variacion_estocastica(distribucion, param1, param2=None):
    """
    Genera una variación estocástica según la distribución especificada.
    """
    if pd.isna(param1) or (param2 is not None and pd.isna(param2)):
        return 0

    if distribucion == 'Poisson':
        return np.random.poisson(lam=param1)

    elif distribucion == 'Normal':
        if param2 is None:
            return 0
        return max(0, int(np.random.normal(loc=param1, scale=param2)))

    elif distribucion == 'Uniforme':
        if param2 is None:
            return 0
        return np.random.randint(int(param1), int(param2) + 1)

    elif distribucion in ['Log-Normal', 'LogNormal']:
        if param2 is None:
            return 0
        return max(0, int(np.random.lognormal(mean=param1, sigma=param2)))

    elif distribucion == 'Gamma':
        if param2 is None:
            return 0
        return max(0, int(np.random.gamma(shape=param1, scale=param2)))

    elif distribucion == 'Constante':
        return int(param1)

    else:
        return 0


DEMANDA DIGITAL

In [108]:
# === FUNCIÓN PRINCIPAL PARA UN DÍA ===
def generar_demanda_digital_estocastica(ventas_zona_dia):
    demanda_estocastica = []

    for _, row in ventas_zona_dia.iterrows():
        id_zona = row['id_zona']
        id_producto = row['id_producto']
        base = row['cantidad']

        if id_producto not in resumen_zonas_con_param.index:
            total = base
        else:
            fila = resumen_zonas_con_param.loc[id_producto]
            dist = fila['mejor_ajuste']
            p1 = fila['parametro1']
            p2 = fila['parametro2'] if not pd.isna(fila['parametro2']) else None
            variacion = generar_variacion_estocastica(dist, p1, p2)
            total = max(0, int(base + variacion))

        demanda_estocastica.append({
            'id_zona': id_zona,
            'id_producto': id_producto,
            'venta_digital': total
        })

    return pd.DataFrame(demanda_estocastica)


Generar demanda digital para los 40 días

In [109]:
import os
import pandas as pd

def generar_ventas_zona_estocasticas(ruta_base_original='venta_zona', carpeta_salida='ventas_zona_estocasticas', dias=40):
    """
    Genera archivos estocásticos tipo venta_zona_dia_X.csv en base a archivos de ventas digitales originales.
    
    - ruta_base_original: nombre base de los archivos originales (sin _n_20250115.csv)
    - carpeta_salida: carpeta donde guardar los nuevos archivos
    - dias: cantidad de días a simular (por defecto: 40)
    """
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)

    for dia in range(1, dias + 1):
        archivo_original = f"{ruta_base_original}_{dia}_20250115.csv"
        if not os.path.exists(archivo_original):
            print(f"⚠️ Archivo no encontrado: {archivo_original}")
            continue

        ventas_zona_dia = pd.read_csv(archivo_original)
        ventas_zona_dia.rename(columns={'venta_digital': 'cantidad'}, inplace=True)

        # Aplicar variación estocástica
        ventas_estocasticas = generar_demanda_digital_estocastica(ventas_zona_dia)

        # Guardar archivo con el nuevo nombre
        nombre_salida = os.path.join(carpeta_salida, f"venta_zona_dia_{dia}.csv")
        ventas_estocasticas.to_csv(nombre_salida, index=False)

        print(f"✅ Generado: {nombre_salida}")



TIENDAS MAPEADAS EN ZONAS

In [110]:
zonas = pd.read_csv('zonas_20250115.csv')  # Contiene columna tienda_zona
ventas_zona = ventas_zona.merge(zonas[['id_zona', 'tienda_zona']], on='id_zona')
ventas_zona.rename(columns={'tienda_zona': 'id_tienda'}, inplace=True)


## Simulación de Consumo y Reabastecimiento Día a Día (Caso Base)

In [111]:
def simulacion_caso_base_optimizada(distribuciones_tienda, distribuciones_zona,
                                    ventas_tienda, ventas_zona, reorden,
                                    dias_simulados=10, frecuencia_reabastecimiento=5,
                                    costo_inventario_unitario=3.733):
    # Diccionarios rápidos para generar variación
    dist_tienda_dict = distribuciones_tienda.set_index('id_producto')[['mejor_ajuste', 'parametro1', 'parametro2']].to_dict('index')
    dist_zona_dict = distribuciones_zona.set_index('id_producto')[['mejor_ajuste', 'parametro1', 'parametro2']].to_dict('index')

    # Inicializar stock
    stock = reorden[['id_tienda', 'id_producto', 'reorden']].copy()
    stock['stock_actual'] = stock['reorden']
    stock.set_index(['id_tienda', 'id_producto'], inplace=True)

    registro_dia_a_dia = []
    costos_diarios = []
    stock_dia = []

    for dia in range(1, dias_simulados + 1):
        # Filtrar demanda física y digital
        vt = ventas_tienda[ventas_tienda['dia'] == dia][['id_tienda', 'id_producto', 'cantidad']]
        vz = ventas_zona[ventas_zona['dia'] == dia][['id_tienda', 'id_producto', 'cantidad']]
        demanda = pd.concat([vt, vz], ignore_index=True)

        # Agrupar demanda total
        demanda_total = demanda.groupby(['id_tienda', 'id_producto'])['cantidad'].sum().reset_index()

        # Generar variación estocástica
        demanda_total['variacion'] = demanda_total.apply(lambda row: generar_variacion_estocastica(
            *(dist_tienda_dict.get(row['id_producto']) or dist_zona_dict.get(row['id_producto']) or ('Constante', 0, None))
        ), axis=1)

        demanda_total['cantidad_ajustada'] = (demanda_total['cantidad'] + demanda_total['variacion']).clip(lower=0).astype(int)

        # Atender demanda
        for _, row in demanda_total.iterrows():
            clave = (row['id_tienda'], row['id_producto'])
            if clave in stock.index:
                disponible = stock.at[clave, 'stock_actual']
                atendido = min(disponible, row['cantidad_ajustada'])
                no_atendido = row['cantidad_ajustada'] - atendido
                stock.at[clave, 'stock_actual'] -= atendido

                registro_dia_a_dia.append({
                    'dia': dia,
                    'id_tienda': row['id_tienda'],
                    'id_producto': row['id_producto'],
                    'demanda': row['cantidad_ajustada'],
                    'atendido': atendido,
                    'no_atendido': no_atendido,
                    'stock_restante': stock.at[clave, 'stock_actual']
                })

        # Guardar snapshot de stock
        df_stock_dia = stock.reset_index().copy()
        df_stock_dia['dia'] = dia
        stock_dia.append(df_stock_dia)

        # Calcular costo diario
        stock_total = stock['stock_actual'].sum()
        costo_dia = stock_total * costo_inventario_unitario
        costos_diarios.append(costo_dia)

        # Reposición
        if dia % frecuencia_reabastecimiento == 0:
            stock['stock_actual'] += stock['reorden'] - stock['stock_actual']

    # Construir resultados completos
    resultados_df = pd.DataFrame(registro_dia_a_dia)
    df_stock = pd.concat(stock_dia, ignore_index=True)

    # === EXPORTAR ARCHIVOS POR DÍA (formato limpio sin índice) ===
    os.makedirs("salidas_stock_por_dia", exist_ok=True)
    os.makedirs("salidas_demanda_online_por_dia", exist_ok=True)

    for dia in range(1, dias_simulados + 1):
        # Extraer demanda ONLINE desde resultados (filtrando por día y por tienda presente en ventas_zona)
        df_online = resultados_df[
            (resultados_df['dia'] == dia) & 
            (resultados_df['id_tienda'].isin(ventas_zona['id_tienda'].unique()))
        ][['id_tienda', 'id_producto', 'demanda', 'dia']].copy()
    
        df_online.rename(columns={'demanda': 'cantidad'}, inplace=True)
        df_online.to_csv(f"salidas_demanda_online_por_dia/demanda_online_{dia}.csv", index=False)

        # Stock diario
        df_stock[df_stock["dia"] == dia].to_csv(f"salidas_stock_por_dia/stock_dia_{dia}.csv", index=False)

    # Exportar costos diarios
    pd.DataFrame({
        "dia": list(range(1, dias_simulados + 1)),
        "costo_inventario": costos_diarios
    }).to_csv("costo_diario.csv", index=False)

    # Métricas finales
    demanda_total = resultados_df['demanda'].sum()
    atendido_total = resultados_df['atendido'].sum()
    no_atendido_total = resultados_df['no_atendido'].sum()
    costo_total = sum(costos_diarios)

    return {
        'nivel_servicio': atendido_total / demanda_total if demanda_total > 0 else 0,
        'costo_total': costo_total,
        'costo_diario_promedio': costo_total / dias_simulados,
        'demanda_no_atendida_total': no_atendido_total,
        'costos_diarios': costos_diarios,
        'registro_dia_a_dia': resultados_df,
        'stock_diario': df_stock,
        'top_10_quiebres': resultados_df.groupby('id_producto')['no_atendido']
                                        .sum().sort_values(ascending=False).head(10).reset_index()
    }



In [112]:
# Cargar archivos necesarios
distribuciones_tienda = pd.read_excel('resumen_tiendas_con_parametros.xlsx')
distribuciones_zona = pd.read_excel('resumen_zonas_con_parametros.xlsx')

# Ejecutar simulación del caso base
resultados = simulacion_caso_base_optimizada(
    distribuciones_tienda=distribuciones_tienda,
    distribuciones_zona=distribuciones_zona,
    ventas_tienda=ventas_tienda,
    ventas_zona=ventas_zona,
    reorden=reorden,
    dias_simulados=10,
    frecuencia_reabastecimiento=5,
    costo_inventario_unitario=3.733
)

# Mostrar KPIs
print(f"Nivel de servicio: {resultados['nivel_servicio']:.2%}")
print(f"Costo total de inventario: ${resultados['costo_total']:,.0f}")
print(f"Costo diario promedio: ${resultados['costo_diario_promedio']:,.2f}")
print(f"Demanda no atendida total: {resultados['demanda_no_atendida_total']:,.0f}")
print("\nTop 10 productos con más unidades no atendidas:")
display(resultados['top_10_quiebres'])

# Acceder a los DataFrames si los necesitas guardar o analizar:
registro = resultados['registro_dia_a_dia']
stock_diario = resultados['stock_diario']
costos_dia = resultados['costos_diarios']


Nivel de servicio: 99.90%
Costo total de inventario: $3,521,674,176
Costo diario promedio: $352,167,417.57
Demanda no atendida total: 457,391

Top 10 productos con más unidades no atendidas:


,id_producto,no_atendido
0,542,4119
1,73,3909
2,559,3702
3,1007,3409
4,705,3288
5,713,2959
6,437,2885
7,276,2826
8,569,2772
9,218,2764


In [113]:
### registro día a día ###
display(registro.head())  # Ver las primeras 5 filas


,dia,id_tienda,id_producto,demanda,atendido,no_atendido,stock_restante
0,1,1,1,47,47,0,193
1,1,1,2,2088,2088,0,8917
2,1,1,3,2405,2405,0,9510
3,1,1,4,3887,3887,0,15293
4,1,1,5,134,134,0,551


In [114]:
### Stock por producto-tienda-día ###
display(stock_diario.head())


,id_tienda,id_producto,reorden,stock_actual,dia
0,1,1,240,193,1
1,1,2,11005,8917,1
2,1,3,11915,9510,1
3,1,4,19180,15293,1
4,1,5,685,551,1


In [115]:
display(stock_diario[stock_diario["dia"] == 3])


,id_tienda,id_producto,reorden,stock_actual,dia
34476,1,1,240,99,3
34477,1,2,11005,4658,3
34478,1,3,11915,4888,3
34479,1,4,19180,7568,3
34480,1,5,685,290,3
...,...,...,...,...,...
51709,17,1010,7280,3066,3
51710,17,1011,345,145,3
51711,17,1012,7095,3098,3
51712,17,1013,1305,610,3


In [116]:
### lista de costos diarios ###
print("Costos diarios (primeros 10 días):")
print(costos_dia[:10])

Costos diarios (primeros 10 días):
[683875487.303, 518042796.982, 352193830.781, 186343625.224, 20501774.121, 683869723.551, 517949535.443, 352112992.666, 186238582.337, 20545827.254]


## Simulación con Optuna

In [117]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [118]:
def simulacion_para_optuna(distribuciones_tienda, distribuciones_zona,
                            ventas_tienda, ventas_zona, reorden,
                            dias_simulados, frecuencia_reabastecimiento,
                            costo_inventario_unitario, reorden_multiplicador):

    reorden_mod = reorden.copy()
    reorden_mod['reorden'] = (reorden_mod['reorden'] * reorden_multiplicador).clip(lower=1).astype(int)

    dist_tienda_dict = distribuciones_tienda.set_index('id_producto')[['mejor_ajuste', 'parametro1', 'parametro2']].to_dict('index')
    dist_zona_dict = distribuciones_zona.set_index('id_producto')[['mejor_ajuste', 'parametro1', 'parametro2']].to_dict('index')

    stock = reorden_mod[['id_tienda', 'id_producto', 'reorden']].copy()
    stock['stock_actual'] = stock['reorden']
    stock.set_index(['id_tienda', 'id_producto'], inplace=True)

    costos_diarios = []

    for dia in range(1, dias_simulados + 1):
        vt = ventas_tienda[ventas_tienda['dia'] == dia][['id_tienda', 'id_producto', 'cantidad']]
        vz = ventas_zona[ventas_zona['dia'] == dia][['id_tienda', 'id_producto', 'cantidad']]
        demanda = pd.concat([vt, vz], ignore_index=True)
        demanda_total = demanda.groupby(['id_tienda', 'id_producto'])['cantidad'].sum().reset_index()

        demanda_total['variacion'] = demanda_total.apply(lambda row: generar_variacion_estocastica(
            *(dist_tienda_dict.get(row['id_producto']) or dist_zona_dict.get(row['id_producto']) or ('Constante', 0, None))
        ), axis=1)
        demanda_total['cantidad_ajustada'] = (demanda_total['cantidad'] + demanda_total['variacion']).clip(lower=0).astype(int)

        for _, row in demanda_total.iterrows():
            clave = (row['id_tienda'], row['id_producto'])
            if clave in stock.index:
                atendido = min(stock.at[clave, 'stock_actual'], row['cantidad_ajustada'])
                stock.at[clave, 'stock_actual'] -= atendido

        costo_dia = stock['stock_actual'].sum() * costo_inventario_unitario
        costos_diarios.append(costo_dia)

        if dia % frecuencia_reabastecimiento == 0:
            stock['stock_actual'] += stock['reorden'] - stock['stock_actual']

    return sum(costos_diarios)


In [119]:
import optuna

def objetivo(trial):
    # Optuna propondrá valores para estos dos hiperparámetros
    frecuencia_reabastecimiento = trial.suggest_int("frecuencia_reabastecimiento", 2, 10)
    reorden_multiplicador = trial.suggest_float("reorden_multiplicador", 0.5, 2.0)

    # Ejecutamos la simulación con esos parámetros propuestos
    costo_total = simulacion_para_optuna(
        distribuciones_tienda=distribuciones_tienda,
        distribuciones_zona=distribuciones_zona,
        ventas_tienda=ventas_tienda,
        ventas_zona=ventas_zona,
        reorden=reorden,
        dias_simulados=10,
        frecuencia_reabastecimiento=frecuencia_reabastecimiento,
        costo_inventario_unitario=3.733,
        reorden_multiplicador=reorden_multiplicador
    )

    return costo_total  # Esto es lo que Optuna quiere minimizar



In [120]:
### Optuna hace varias pruebas y encuentra la mejor combinación.

# Crear un estudio de minimización (queremos minimizar el costo total)
estudio = optuna.create_study(direction="minimize")

# Ejecutar la optimización (prueba con 20 combinaciones distintas)
estudio.optimize(objetivo, n_trials=20)

[I 2025-06-15 04:51:38,523] A new study created in memory with name: no-name-5a74fb0f-f921-47c9-9278-e573bc0463ab
[I 2025-06-15 04:52:22,820] Trial 0 finished with value: 8985536475.519001 and parameters: {'frecuencia_reabastecimiento': 5, 'reorden_multiplicador': 1.6432044701746567}. Best is trial 0 with value: 8985536475.519001.
[I 2025-06-15 04:53:01,064] Trial 1 finished with value: 1963510169.071 and parameters: {'frecuencia_reabastecimiento': 4, 'reorden_multiplicador': 0.6553369108919034}. Best is trial 1 with value: 1963510169.071.
[I 2025-06-15 04:53:42,266] Trial 2 finished with value: 3912742083.4999995 and parameters: {'frecuencia_reabastecimiento': 6, 'reorden_multiplicador': 1.0540321152871586}. Best is trial 1 with value: 1963510169.071.
[I 2025-06-15 04:54:22,525] Trial 3 finished with value: 4583335078.979 and parameters: {'frecuencia_reabastecimiento': 4, 'reorden_multiplicador': 0.9888225791446361}. Best is trial 1 with value: 1963510169.071.
[I 2025-06-15 04:55:02,6

In [121]:
print(" Mejor combinación encontrada por Optuna:")
print(f" Frecuencia de reabastecimiento: {estudio.best_params['frecuencia_reabastecimiento']}")
print(f" Multiplicador de reorden: {estudio.best_params['reorden_multiplicador']:.2f}")
print(f" Costo total mínimo encontrado: ${estudio.best_value:,.0f}")


 Mejor combinación encontrada por Optuna:
 Frecuencia de reabastecimiento: 8
 Multiplicador de reorden: 0.52
 Costo total mínimo encontrado: $783,115,161


### Resultados:
- Frecuencia de reabastecimiento = 10 días
- Multiplicador de reorden = 0.52 (reduce el stock base a casi la mitad)
- Costo total =	$380,313,967
El valor que entrega Optuna es el costo total de inventario para los 10 días simulados.


In [122]:
### SIMULACIÓN CON LOS PARAMETROS DE OPTUNA ###
# Crear nueva política con lo que encontró Optuna
reorden_optimo = reorden.copy()
reorden_optimo['reorden'] = (reorden_optimo['reorden'] * estudio.best_params['reorden_multiplicador']).astype(int)

# Simular con la política óptima
resultados_optimo = simulacion_caso_base_optimizada(
    distribuciones_tienda=distribuciones_tienda,
    distribuciones_zona=distribuciones_zona,
    ventas_tienda=ventas_tienda,
    ventas_zona=ventas_zona,
    reorden=reorden_optimo,
    dias_simulados=10,
    frecuencia_reabastecimiento=estudio.best_params['frecuencia_reabastecimiento'],
    costo_inventario_unitario=3.733
)


In [123]:
# KPIs
print(f"Nivel de servicio (óptimo): {resultados_optimo['nivel_servicio']:.2%}")
print(f"Costo total inventario (óptimo): ${resultados_optimo['costo_total']:,.0f}")
print(f"Demanda no atendida total: {resultados_optimo['demanda_no_atendida_total']:,}")


Nivel de servicio (óptimo): 46.81%
Costo total inventario (óptimo): $783,113,294
Demanda no atendida total: 236,558,103


- Claramenente se verifica que Optuna solo minimiza una métrica (el costo total), pero no considera que queremos tambíen un buen nivel de servicio. 

- Entonces, es normal que encuentre un costo mucho más bajo... a costa de dejar gran parte de la demanda sin atender.

- Por lo que ahora hay que buscar que combinación de ambos factores es la indicada.

In [124]:
### Minimizar el costo total sin permitir que el nivel de servicio baje de cierto umbral (90%) ###
def objetivo_con_penalizacion(trial):
    frecuencia_reabastecimiento = trial.suggest_int("frecuencia_reabastecimiento", 2, 10)
    reorden_multiplicador = trial.suggest_float("reorden_multiplicador", 0.5, 2.0)

    # Ejecutar simulación rápida
    costo_total = simulacion_para_optuna(
        distribuciones_tienda=distribuciones_tienda,
        distribuciones_zona=distribuciones_zona,
        ventas_tienda=ventas_tienda,
        ventas_zona=ventas_zona,
        reorden=reorden,
        dias_simulados=10,
        frecuencia_reabastecimiento=frecuencia_reabastecimiento,
        costo_inventario_unitario=3.733,
        reorden_multiplicador=reorden_multiplicador
    )

    # Calcular nivel de servicio usando simulación completa pero sin guardar archivos
    reorden_modificado = reorden.copy()
    reorden_modificado['reorden'] = (reorden_modificado['reorden'] * reorden_multiplicador).astype(int)
    resultados_temporales = simulacion_caso_base_optimizada(
        distribuciones_tienda=distribuciones_tienda,
        distribuciones_zona=distribuciones_zona,
        ventas_tienda=ventas_tienda,
        ventas_zona=ventas_zona,
        reorden=reorden_modificado,
        dias_simulados=10,
        frecuencia_reabastecimiento=frecuencia_reabastecimiento,
        costo_inventario_unitario=3.733
    )

    nivel_servicio = resultados_temporales['nivel_servicio']

    # Penalizar si el nivel de servicio es menor a 90%
    if nivel_servicio < 0.90:
        penalizacion = 1e9  # 1 billón
        return costo_total + penalizacion
    else:
        return costo_total


In [125]:
estudio_penalizado = optuna.create_study(direction="minimize")
estudio_penalizado.optimize(objetivo_con_penalizacion, n_trials=20)


[I 2025-06-15 05:05:40,606] A new study created in memory with name: no-name-757bd4dd-ac97-4024-a714-2fc1ef18797c
[I 2025-06-15 05:07:11,005] Trial 0 finished with value: 3973960827.1859994 and parameters: {'frecuencia_reabastecimiento': 6, 'reorden_multiplicador': 1.062036005849868}. Best is trial 0 with value: 3973960827.1859994.
[W 2025-06-15 05:07:53,780] Trial 1 failed with parameters: {'frecuencia_reabastecimiento': 7, 'reorden_multiplicador': 0.573240736162987} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Usuario\AppData\Local\Temp\ipykernel_13604\3330965552.py", line 22, in objetivo_con_penalizacion
    resultados_temporales = simulacion_caso_base_optimizada(
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


KeyboardInterrupt: 

In [ ]:
print("🏆 Mejor política penalizada:")
print(f"📌 Frecuencia: {estudio_penalizado.best_params['frecuencia_reabastecimiento']}")
print(f"📌 Multiplicador reorden: {estudio_penalizado.best_params['reorden_multiplicador']:.2f}")
print(f"💰 Costo total (penalizado): ${estudio_penalizado.best_value:,.0f}")

🏆 Mejor política penalizada:
📌 Frecuencia: 2
📌 Multiplicador reorden: 0.51
💰 Costo total (penalizado): $1,822,461,530


In [ ]:
### SIMULACIÓN CON LOS PARAMETROS DE OPTUNA + PENALIZACION (1) ###
# Crear política usando los mejores parámetros encontrados
reorden_penalizado = reorden.copy()
reorden_penalizado['reorden'] = (reorden_penalizado['reorden'] * estudio_penalizado.best_params['reorden_multiplicador']).astype(int)

# Ejecutar simulación final con esa política
resultados_penalizado = simulacion_caso_base_optimizada(
    distribuciones_tienda=distribuciones_tienda,
    distribuciones_zona=distribuciones_zona,
    ventas_tienda=ventas_tienda,
    ventas_zona=ventas_zona,
    reorden=reorden_penalizado,
    dias_simulados=10,
    frecuencia_reabastecimiento=estudio_penalizado.best_params['frecuencia_reabastecimiento'],
    costo_inventario_unitario=3.733
)

In [ ]:
### KPIS ###
print(f"📦 Nivel de servicio (penalizado): {resultados_penalizado['nivel_servicio']:.2%}")
print(f"💰 Costo total inventario (penalizado): ${resultados_penalizado['costo_total']:,.0f}")
print(f"🚫 Demanda no atendida total: {resultados_penalizado['demanda_no_atendida_total']:,}")


📦 Nivel de servicio (penalizado): 100.00%
💰 Costo total inventario (penalizado): $1,822,459,664
🚫 Demanda no atendida total: 19


Politica con:
- Frecuencia de reabastecimiento = 2 días
- Reorden reducido al 51% del original
- Y sin sacrificar nivel de servicio. Esto sugiere que el modelo original tenía un exceso de stock innecesario que podías evitar con una política más ajustada.

Guardar los archivos como csv

In [ ]:
### GUARDAR LOS ARCHIBOS COMO CSV ###
def exportar_resultados_simulacion(resultados_df, df_stock, costos_diarios, ventas_zona, dias_simulados, sufijo="base"): #ESTE SERÍA EL BASE X DEFAULT
    """
    Guarda los archivos .csv de stock, demanda online y costo diario con un sufijo identificador.
    """
    carpeta_stock = f"salidas_stock_por_dia_{sufijo}"
    carpeta_online = f"salidas_demanda_online_por_dia_{sufijo}"
    os.makedirs(carpeta_stock, exist_ok=True)
    os.makedirs(carpeta_online, exist_ok=True)

    for dia in range(1, dias_simulados + 1):
        # Demanda online simulada por día
        df_online = resultados_df[
            (resultados_df['dia'] == dia) & 
            (resultados_df['id_tienda'].isin(ventas_zona['id_tienda'].unique()))
        ][['id_tienda', 'id_producto', 'demanda', 'dia']].copy()
        df_online.rename(columns={'demanda': 'cantidad'}, inplace=True)
        df_online.to_csv(f"{carpeta_online}/demanda_online_{dia}_{sufijo}.csv", index=False)

        # Stock diario por día
        df_stock[df_stock["dia"] == dia].to_csv(f"{carpeta_stock}/stock_dia_{dia}_{sufijo}.csv", index=False)

    # Costos diarios consolidados
    pd.DataFrame({
        "dia": list(range(1, dias_simulados + 1)),
        "costo_inventario": costos_diarios
    }).to_csv(f"costo_diario_{sufijo}.csv", index=False)

    print(f"✅ Resultados exportados con sufijo: {sufijo}")

In [ ]:
exportar_resultados_simulacion(
    resultados_df=resultados_optimo['registro_dia_a_dia'],
    df_stock=resultados_optimo['stock_diario'],
    costos_diarios=resultados_optimo['costos_diarios'],
    ventas_zona=ventas_zona,
    dias_simulados=10,
    sufijo="optimo"
)


✅ Resultados exportados con sufijo: optimo


In [ ]:
exportar_resultados_simulacion(
    resultados_df=resultados_penalizado['registro_dia_a_dia'],
    df_stock=resultados_penalizado['stock_diario'],
    costos_diarios=resultados_penalizado['costos_diarios'],
    ventas_zona=ventas_zona,
    dias_simulados=10,
    sufijo="penalizado"
)


✅ Resultados exportados con sufijo: penalizado


### RESUMEN RESULTADOS
| Política                 | Nivel Servicio | Costo Total             | Demanda No Atendida  |
| ------------------------ | -------------- | ----------------------- | -------------------- |
| Base                     | 99.9%          | \$3.52 mil millones     | 457,391 unidades     |
| Óptima (sin restricción) | 26.4%          | \$380 millones          | 327 millones         |
| Penalizada               | 100.0%         | **\$1.82 mil millones** | **solo 19 unidades** |


- Política penalizada logra:
    - Mismo (o mejor) nivel de servicio que la base
    - Casi 50% de ahorro en costos
    - Prácticamente cero quiebres de stock
- La política “óptima” sin restricciones bajó costos... pero a costa de perder el 73% de la demanda.



In [ ]:
### simulacion considerando nivel de servicio por tienda y por producto ###
def simulacion_con_kpis_desglosados(distribuciones_tienda, distribuciones_zona,
                                     ventas_tienda, ventas_zona, reorden,
                                     dias_simulados, frecuencia_reabastecimiento,
                                     costo_inventario_unitario, reorden_multiplicador):
    """
    Ejecuta una simulación con demanda estocástica y calcula:
    - Costo total de inventario
    - Nivel de servicio global
    - Nivel de servicio por tienda
    - Nivel de servicio por producto
    """

    import pandas as pd

    reorden_mod = reorden.copy()
    reorden_mod['reorden'] = (reorden_mod['reorden'] * reorden_multiplicador).clip(lower=1).astype(int)

    dist_tienda_dict = distribuciones_tienda.set_index('id_producto')[['mejor_ajuste', 'parametro1', 'parametro2']].to_dict('index')
    dist_zona_dict = distribuciones_zona.set_index('id_producto')[['mejor_ajuste', 'parametro1', 'parametro2']].to_dict('index')

    stock = reorden_mod[['id_tienda', 'id_producto', 'reorden']].copy()
    stock['stock_actual'] = stock['reorden']
    stock.set_index(['id_tienda', 'id_producto'], inplace=True)

    costos_diarios = []
    registro = []

    for dia in range(1, dias_simulados + 1):
        vt = ventas_tienda[ventas_tienda['dia'] == dia][['id_tienda', 'id_producto', 'cantidad']]
        vz = ventas_zona[ventas_zona['dia'] == dia][['id_tienda', 'id_producto', 'cantidad']]
        demanda = pd.concat([vt, vz], ignore_index=True)
        demanda_total = demanda.groupby(['id_tienda', 'id_producto'])['cantidad'].sum().reset_index()

        demanda_total['variacion'] = demanda_total.apply(lambda row: generar_variacion_estocastica(
            *(dist_tienda_dict.get(row['id_producto']) or dist_zona_dict.get(row['id_producto']) or ('Constante', 0, None))
        ), axis=1)
        demanda_total['cantidad_ajustada'] = (demanda_total['cantidad'] + demanda_total['variacion']).clip(lower=0).astype(int)

        for _, row in demanda_total.iterrows():
            clave = (row['id_tienda'], row['id_producto'])
            if clave in stock.index:
                atendido = min(stock.at[clave, 'stock_actual'], row['cantidad_ajustada'])
                stock.at[clave, 'stock_actual'] -= atendido

                registro.append({
                    'id_tienda': row['id_tienda'],
                    'id_producto': row['id_producto'],
                    'demanda': row['cantidad_ajustada'],
                    'atendido': atendido
                })

        costo_dia = stock['stock_actual'].sum() * costo_inventario_unitario
        costos_diarios.append(costo_dia)

        if dia % frecuencia_reabastecimiento == 0:
            stock['stock_actual'] += stock['reorden'] - stock['stock_actual']

    df_registro = pd.DataFrame(registro)

    # Cálculo de KPIs
    nivel_global = df_registro['atendido'].sum() / df_registro['demanda'].sum()

    nivel_por_tienda = df_registro.groupby('id_tienda').apply(
        lambda g: g['atendido'].sum() / g['demanda'].sum()
    ).reset_index(name='nivel_servicio_tienda')

    nivel_por_producto = df_registro.groupby('id_producto').apply(
        lambda g: g['atendido'].sum() / g['demanda'].sum()
    ).reset_index(name='nivel_servicio_producto')

    return sum(costos_diarios), nivel_global, nivel_por_tienda, nivel_por_producto, df_registro


In [ ]:
costo_total, nivel_global, nivel_tienda, nivel_producto, registro = simulacion_con_kpis_desglosados(
    distribuciones_tienda=distribuciones_tienda,
    distribuciones_zona=distribuciones_zona,
    ventas_tienda=ventas_tienda,
    ventas_zona=ventas_zona,
    reorden=reorden,
    dias_simulados=10,
    frecuencia_reabastecimiento=5,
    costo_inventario_unitario=3.733,
    reorden_multiplicador=1.0
)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_13604\3744659477.py:64: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nivel_por_tienda = df_registro.groupby('id_tienda').apply(
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13604\3744659477.py:68: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nivel_por_producto = df_registro.groupby('id_producto').apply(


In [ ]:
print(f"🔎 Nivel de servicio global: {nivel_global:.2%}")
print(f"💰 Costo total de inventario: ${costo_total:,.0f}")
print("\n✅ Nivel de servicio por tienda:")
print(nivel_tienda)

print("\n✅ Nivel de servicio por producto:")
print(nivel_producto)


🔎 Nivel de servicio global: 99.90%
💰 Costo total de inventario: $3,521,676,042

✅ Nivel de servicio por tienda:
    id_tienda  nivel_servicio_tienda
0           1               0.998788
1           2               0.997338
2           3               0.998083
3           4               0.998619
4           5               0.999493
5           6               0.999363
6           7               0.999082
7           8               0.998721
8           9               0.999255
9          10               0.999034
10         11               0.999409
11         12               0.998746
12         13               0.997820
13         14               0.998072
14         15               0.998665
15         16               0.999040
16         17               0.998922

✅ Nivel de servicio por producto:
      id_producto  nivel_servicio_producto
0               1                 1.000000
1               2                 0.999954
2               3                 0.999581
3              

In [ ]:
from IPython.display import display

resumen = []

for i in range(1, 41):
    archivo = f"venta_zona_{i}_20250115.csv"
    try:
        df = pd.read_csv(archivo)

        max_val = df['venta_digital'].max()
        min_val = df['venta_digital'].min()
        conteo_cero = (df['venta_digital'] == 0).sum()

        resumen.append({
            'archivo': archivo,
            'max_venta_digital': max_val,
            'min_venta_digital': min_val,
            'ventas_digital_cero': conteo_cero
        })
    except Exception as e:
        resumen.append({
            'archivo': archivo,
            'error': str(e)
        })

df_resumen_ventas = pd.DataFrame(resumen)
display(df_resumen_ventas)

,archivo,max_venta_digital,min_venta_digital,ventas_digital_cero
0,venta_zona_1_20250115.csv,193,0,64981
1,venta_zona_2_20250115.csv,196,0,64901
2,venta_zona_3_20250115.csv,210,0,64889
3,venta_zona_4_20250115.csv,179,0,64973
4,venta_zona_5_20250115.csv,198,0,64839
5,venta_zona_6_20250115.csv,224,0,65102
6,venta_zona_7_20250115.csv,207,0,64858
7,venta_zona_8_20250115.csv,196,0,65048
8,venta_zona_9_20250115.csv,196,0,65044
9,venta_zona_10_20250115.csv,195,0,65214


In [ ]:
from IPython.display import display

resumen_2 = []

for i in range(1, 11):
    archivo_2 = f"salidas_demanda_online_por_dia/demanda_online_{i}.csv"
    try:
        df = pd.read_csv(archivo_2)

        max_val = df['cantidad'].max()
        min_val = df['cantidad'].min()
        conteo_cero = (df['cantidad'] == 0).sum()

        resumen_2.append({
            'archivo': archivo_2,
            'max_demanda_digital': max_val,
            'min_demanda_digital': min_val,
            'demanda_online_cero': conteo_cero
        })
    except Exception as e:
        resumen_2.append({
            'archivo': archivo_2,
            'error': str(e)
        })

df_resumen_ventas_2 = pd.DataFrame(resumen_2)
display(df_resumen_ventas_2)

,archivo,max_demanda_digital,min_demanda_digital,demanda_online_cero
0,salidas_demanda_online_por_dia/demanda_online_...,12697,0,79
1,salidas_demanda_online_por_dia/demanda_online_...,12602,0,71
2,salidas_demanda_online_por_dia/demanda_online_...,12606,0,76
3,salidas_demanda_online_por_dia/demanda_online_...,12803,0,76
4,salidas_demanda_online_por_dia/demanda_online_...,12515,0,64
5,salidas_demanda_online_por_dia/demanda_online_...,12651,0,76
6,salidas_demanda_online_por_dia/demanda_online_...,12681,0,73
7,salidas_demanda_online_por_dia/demanda_online_...,12576,0,80
8,salidas_demanda_online_por_dia/demanda_online_...,12573,0,73
9,salidas_demanda_online_por_dia/demanda_online_...,12904,0,62


In [127]:
# Reimportar después del reinicio de entorno
import numpy as np
import pandas as pd
from scipy.stats import norm, lognorm, gamma, poisson

# Función actualizada: generación de variación estocástica acotada como factor multiplicador
def generar_variacion_estocastica_2(distribucion, param1, param2=None):
    """
    Genera un factor multiplicador estocástico entre ~0.7 y ~1.3
    según la distribución ajustada del producto.
    """
    # Si los parámetros son inválidos o faltantes, retornar 1.0 (sin variación)
    if pd.isna(param1) or (param2 is not None and pd.isna(param2)):
        return 1.0

    if distribucion == "Poisson":
        variacion = np.random.poisson(lam=param1) / param1 if param1 > 0 else 1.0
    elif distribucion == "Normal":
        if param2 is None or param2 <= 0:
            return 1.0
        variacion = np.random.normal(loc=param1, scale=param2) / param1 if param1 > 0 else 1.0
    elif distribucion == "Uniforme":
        if param2 is None:
            return 1.0
        variacion = np.random.uniform(low=param1, high=param2) / ((param1 + param2) / 2)
    elif distribucion in ["Log-Normal", "LogNormal"]:
        if param2 is None or param2 <= 0:
            return 1.0
        variacion = np.random.lognormal(mean=param1, sigma=param2) / np.exp(param1)
    elif distribucion == "Gamma":
        if param2 is None or param2 <= 0:
            return 1.0
        variacion = np.random.gamma(shape=param1, scale=param2) / (param1 * param2)
    else:
        return 1.0  # Distribución desconocida

    # Acotar la variación para evitar valores extremos
    variacion = max(0.7, min(variacion, 1.3))

    return variacion


In [129]:
def generar_demanda_digital_estocastica_2(ventas_zona_dia, resumen_parametros):
    """
    Genera demanda digital estocástica para un DataFrame diario de ventas por zona,
    aplicando un factor multiplicador acotado según la distribución ajustada.
    """
    demanda_estocastica = []

    for _, row in ventas_zona_dia.iterrows():
        id_zona = row['id_zona']
        id_producto = row['id_producto']
        base = row['venta_digital']

        if id_producto not in resumen_parametros.index:
            total = base
        else:
            fila = resumen_parametros.loc[id_producto]
            dist = fila['mejor_ajuste']
            p1 = fila['parametro1']
            p2 = fila['parametro2'] if not pd.isna(fila['parametro2']) else None
            variacion = generar_variacion_estocastica_2(dist, p1, p2)  # <- usa la nueva
            total = max(0, int(base * variacion))

        demanda_estocastica.append({
            'id_zona': id_zona,
            'id_producto': id_producto,
            'venta_digital': total
        })

    return pd.DataFrame(demanda_estocastica)


In [130]:
ventas_zona_dia = pd.read_csv("venta_zona_1_20250115.csv")
resumen_parametros = pd.read_excel("resumen_zonas_con_parametros.xlsx").set_index("id_producto")

df_demanda_estocastica_2 = generar_demanda_digital_estocastica_2(ventas_zona_dia, resumen_parametros)
display(df_demanda_estocastica_2)


,id_zona,id_producto,venta_digital
0,1,1,1
1,1,2,29
2,1,3,48
3,1,4,12
4,1,5,1
...,...,...,...
2129395,2100,1010,24
2129396,2100,1011,1
2129397,2100,1012,9
2129398,2100,1013,2
